#### torchtext 라이브러리로 텍스트 분류 <hr>
- 1단계 : 데이터 전처리 => 숫자형식으로 변환
- 2단계 : 모델 구현

[1-1] 데이터 준비 => 내장 데이터셋 활용
- AGNEWS 데이터셋 반복자 : 레이블(label) + 문장의 튜풀(tuple) 형태

In [2]:
# !pip install torchdata

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.3 MB 656.4 kB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.3 MB 1.0 MB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.3 MB 1.6 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.3 MB 2.0 MB/s eta 0:00:01
   ---------------------- ----------------- 0.8/1.3 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------  1.3/1.3 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/172.0 kB ? eta -:--:--
   --------------------------------------- 172.0/172.0 kB 10.1 MB/s eta 0:00:00


In [8]:
import torchtext
torchtext.__version__

'0.17.2'

In [4]:
import torch
from torchtext.datasets import AG_NEWS

# DataPipe 타입 >>> iterator 타입 형변환
train_iter = iter(AG_NEWS(split = 'train'))

In [5]:
# 데이터 확인 => (label, text), label : 1 ~ 4
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

[2] 데이터 전처리 파이프라인 준비
- 어휘집(vocab), 단어 벡터(word vector), 토크나이저(tokenizer)
- 가공되지 않은 텍스트 문자열에 대한 데이터 처리 빌딩 블록
- 일반적인 NLP 데이터 처리
    - 첫번째 단계 : 가공되지 않은 학습 데이터셋으로 어휘집 생성  
    ==> 토큰 목록 또는 반복자 받는 내장 팩토리 함수 (factory function) : ``bulid_vocab_from_iterator``
    - 사용자는 어휘집에 추가할 특수 기호(special symbol) 전달 가능


In [12]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [13]:
# 토커나이즈 생성
tokenizer = get_tokenizer('basic_english')

# 뉴스 학습 데이터 추출
train_iter = AG_NEWS(split = 'train')

In [14]:
# 토큰 제너레이터 함수 : 데이터 추출하여 토큰화
def yield_tokens(data_iter):
    for _, text in data_iter:
        # 라벨, 텍스트 => 텍스트 토큰화
        yield tokenizer(text)

In [17]:
# 단어사전 생성
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials = ['<unk>'])

# <UNK> 인덱스 0으로 설정
vocab.set_default_index(vocab['<unk>'])

In [18]:
vocab(['<unk>', 'here', 'is', 'an', 'example'])

[0, 475, 21, 30, 5297]

In [19]:
# 텍스트 => 정수 인코딩
text_pipeline = lambda x : vocab(tokenizer(x))

# 레이블 => 정수 인코딩
label_pipeline = lambda x : int(x) - 1

[3] 데이터 배치(batch)와 반복자 생성
- torch.utils.data.DataLoader : getitem(), len() 구현한 맵 형태(map-style)
- collate_fn() : DataLoader로부터 생성된 샘플 배치 함수  
==> 입력 : DataLoader에 배치 크기(batch size)가 있는 배치(batch) 데이터

In [20]:
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 배치 크기만큼 데이터셋 반환 함수
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]

    # 1개씩 뉴스기사, 라벨 추출해서 저장
    for (_label, _text) in batch:
        # 라벨 인코딩 후 저장
        label_list.append(label_pipeline(_label))

        # 텍스트 인코딩 후 저장
        processed_text = torch.tensor(text_pipeline(_text), dtype = torch.int64)
        text_list.append(processed_text)

        # 텍스트 offset 즉, 텍스트 크기/길이 저장
        offsets.append(processed_text.size(0))
    
    label_list = torch.tensor(label_list, dtype = torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim = 0)
    text_list = torch.cat(text_list)

    return label_list.to(device), text_list.to(device), offsets.to(device)

In [21]:
train_iter = AG_NEWS(split = 'train')
dataloader = DataLoader(train_iter,
                        batch_size = 8,
                        shuffle = True,
                        collate_fn = collate_batch)

In [22]:
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)

print(f'num class : {num_class}   vocab size : {vocab_size}')

num class : 4   vocab size : 95811


In [25]:
for labels, texts, offsets in dataloader:
    print(labels, texts, offsets)
    break

tensor([0, 2, 1, 2, 1, 0, 0, 3]) tensor([ 1756,  8246,  2932,  1173,     3,  2090,    66,   117,  4407,    13,
           27,    14,    15,     5,   928,  1756,  4556,  5265,     8,  2287,
         2779,     7,   529,  1173,    10,    56,   167, 49357,    66,   117,
          390,   997,     6,   686,     6,   102,    39,    81,  7033,     1,
         4387,  5702, 18568,   454,  3149,    10,  6869,    92,    13,  4895,
           14,  1146,     1,   640,    13,  4685,    14,    53,  4387,  5702,
            3,     2,    50,    12,     9,   356,  2887,    54,     3,    46,
           72,  2636,  4853,   153,    19,   622,   486,  2178,  6869,    92,
            8,    26,    25,   104,   465,    19,   389,    19,  1284,   139,
            4,   360,     3,   334,   252,   411, 17149,     1,  1453,   444,
         3246,  6353,     4,  1504,    13,    31,    14,    31,    15,   835,
         1453,  2166,   717,   343,  2990,  6353,    57,     4,   478,     6,
        13847,     5,  3750,  2

c:\ProgramData\anaconda3\envs\Torch_NLP38\lib\site-packages\torch\utils\data\datapipes\iter\combining.py:337: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "
